In [2]:
import pandas as pd
import reverse_geocoder as rg
from geopy.geocoders import Nominatim 
from googletrans import Translator, constants
translator = Translator()
geolocator = Nominatim(user_agent="geoapiExercises")

# Data cleaning & merging

In [12]:
def input_org_data():
    e_data = pd.read_csv("https://www.dropbox.com/s/u2xupuhf07y85zh/earthQuake.csv?dl=1")
    density_data = pd.read_csv("https://www.dropbox.com/s/p6ejr4mzx8pjq9m/population_density_long.csv?dl=1")
    population_data=pd.read_csv("https://www.dropbox.com/s/bp5hb61zrui2qo6/population_total_long.csv?dl=1")
    e_data_with_country = pd.read_csv('out_e_data.csv', delimiter=',', encoding="utf-8")
    density_data = pd.read_csv('population_density_long.csv', delimiter=',', encoding="utf-8")
    population_data=pd.read_csv('population_total_long.csv', delimiter=',', encoding="utf-8")

In [14]:
e_data = pd.read_csv("https://www.dropbox.com/s/u2xupuhf07y85zh/earthQuake.csv?dl=1")
print(e_data
    )

             Date      Time  Latitude  Longitude        Type   Depth  \
0      01/02/1965  13:44:18   19.2460   145.6160  Earthquake  131.60   
1      01/04/1965  11:29:49    1.8630   127.3520  Earthquake   80.00   
2      01/05/1965  18:05:58  -20.5790  -173.9720  Earthquake   20.00   
3      01/08/1965  18:49:43  -59.0760   -23.5570  Earthquake   15.00   
4      01/09/1965  13:32:50   11.9380   126.4270  Earthquake   15.00   
...           ...       ...       ...        ...         ...     ...   
23407  12/28/2016  08:22:12   38.3917  -118.8941  Earthquake   12.30   
23408  12/28/2016  09:13:47   38.3777  -118.8957  Earthquake    8.80   
23409  12/28/2016  12:38:51   36.9179   140.4262  Earthquake   10.00   
23410  12/29/2016  22:30:19   -9.0283   118.6639  Earthquake   79.00   
23411  12/30/2016  20:08:28   37.3973   141.4103  Earthquake   11.94   

       Magnitude Magnitude Type            ID  Source Location Source  \
0            6.0             MW  ISCGEM860706  ISCGEM         

In [5]:
full_combined_dataset = pd.read_csv('clean_data.csv', delimiter=',', encoding="utf-8")
print(full_combined_dataset)

       Unnamed: 0  Latitude  Longitude   Depth  Magnitude  Month   Day  \
0               0   19.2460   145.6160  131.60        6.0      1   2.0   
1              39   53.0080  -162.0080   17.80        6.4      2   6.0   
2              44   53.1910  -161.8590   22.50        6.3      2   6.0   
3              57   21.5270   143.0810  340.00        5.6      2  17.0   
4              86   53.2920  -162.3310   20.00        5.7      3  13.0   
...           ...       ...        ...     ...        ...    ...   ...   
18720       23325    6.1142   -82.5906   10.00        5.6     11  12.0   
18721       23359  -35.2021    54.0219   10.00        5.5     12   1.0   
18722       23365   11.0316   -60.9005   31.75        5.9     12   6.0   
18723       23397  -10.1785   160.9149   10.00        6.0     12  20.0   
18724       23398  -10.1549   160.7816   10.38        5.5     12  20.0   

         Year    Country Name  Density  Population  
0      1965.0   United States       21   194303000  
1    

In [ ]:
def cleanEarthquakeData(): 
    e_data[['Month','Day','Year']]=e_data['Date'].str.split('/',-1,expand=True)
    e_data = e_data.drop(['Date','Time','ID','Source','Location Source','Magnitude Source','Status','Type','Magnitude Type'],axis=1)
    print(e_data)

In [ ]:
def clean_combine_pop_density():
    population_data = population_data.drop(population_data.index[population_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
    population_data.rename(columns={'Count': 'Population'}, inplace=True)

    density_data = density_data.drop(density_data.index[density_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
    density_data.rename(columns={'Count': 'Density'}, inplace=True)

    joined = density_data.merge(population_data,
              how='left',
              on =['Country Name', 'Year'],
             )
    joined['Population'] = joined['Population'].fillna(0).astype(int)
    print(joined)
    pop_dense_data = joined


In [ ]:
def find_country(long,lat):
    tuple_input = (long,lat)
    found = rg.search(tuple_input)
    countryName = found[0]['name']
#     countryName = translator.translate(countryName)
    print('**', countryName)
    
    return countryName

def city_to_country(city):
    try:
        loc = geolocator.geocode(city, language='en')
        loc_list = loc.address.split(',')
    except:
        print(city + ': Not found')
        return None
    return loc_list[-1]

In [ ]:
def country_col_edata():
    count = 0
    for i, r, in e_data.iterrows():
        country = city_to_country(find_country(r['Latitude'], r['Longitude']))
        e_data['Country Name'][count] = country
        print(country)
        count +=1
    
    e_data_with_country = e_data_with_country.dropna()
    print(e_data_with_country)
    return e_data_with_country

In [ ]:
def strip_white_space():
    fixed_e_data = e_data_with_country
    fixed_e_data['Country Name'] = fixed_e_data['Country Name'].str.strip()
    g = fixed_e_data['Country Name'][2]
    print(type(g))
    print(g)

def merge_popDense_e_data():
    joined = e_data_with_country.merge(fixed_pop_data,
           how='inner',
          on =['Country Name', 'Year'],
         )
    joined['Population'] = joined['Population'].fillna(0).astype(int)
    joined['Density'] = joined['Density'].fillna(0).astype(int)
    print(joined)


In [15]:
#Clean data
e_data = pd.read_csv("https://www.dropbox.com/s/ey2y94hyfclgl98/clean_data.csv?dl=1")

In [16]:
print(e_data)

       Unnamed: 0  Latitude  Longitude   Depth  Magnitude  Month   Day  \
0               0   19.2460   145.6160  131.60        6.0      1   2.0   
1              39   53.0080  -162.0080   17.80        6.4      2   6.0   
2              44   53.1910  -161.8590   22.50        6.3      2   6.0   
3              57   21.5270   143.0810  340.00        5.6      2  17.0   
4              86   53.2920  -162.3310   20.00        5.7      3  13.0   
...           ...       ...        ...     ...        ...    ...   ...   
18720       23325    6.1142   -82.5906   10.00        5.6     11  12.0   
18721       23359  -35.2021    54.0219   10.00        5.5     12   1.0   
18722       23365   11.0316   -60.9005   31.75        5.9     12   6.0   
18723       23397  -10.1785   160.9149   10.00        6.0     12  20.0   
18724       23398  -10.1549   160.7816   10.38        5.5     12  20.0   

         Year    Country Name  Density  Population  
0      1965.0   United States       21   194303000  
1    